In [ ]:
from hydra_zen import add_conf, add_func_conf, ConfMode, _add_conf #,cs, zen_dry_run, launch_and_instantiate
from hydra_zen import launch
from typing import Any, List
from omegaconf import MISSING
import pandas as pd
%load_ext autoreload 
%autoreload 2
%config Completer.use_jedi = False

: 

# using `add_conf`

## decorate a newly defined class

In [2]:
@add_conf(hydra_recursive=False)
class MyClass:
    def __init__(
        self,
        a:str='dog', 
        b:int=3,
        ):
        self.a = a
        self.b = b
        
    def __repr__(self):
        return f'MyClass({self.a}, {self.b})'
    
MyClass()

MyClass(dog, 3)

In [3]:
# class object has a configuration dataclass attached as .Conf attribute
MyClass.Conf # .Conf attribute is a class object (not an instance)

types.Builds_MyClass

In [4]:
# you can use .Conf just like a class method
# e.g. instantiate a config dataclass with default objects
MyClass.Conf()

Builds_MyClass(_target_='hydra_zen.funcs.zen_processing', _zen_target='__main__.MyClass', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), _recursive_=False, a='dog', b=3, name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_'])

In [5]:
# e.g. instantiate a config dataclass with non default arguments
MyClass.Conf(a='Bloom', b=999)

Builds_MyClass(_target_='hydra_zen.funcs.zen_processing', _zen_target='__main__.MyClass', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), _recursive_=False, a='Bloom', b=999, name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_'])

In [6]:
# instantiate a config dataclass with default objects, then use it to instantiate parent class
MyClass.Conf().instantiate()

MyClass(dog, 3)

In [7]:
# instantiate a config dataclass with non default arguments, then use it to instantiate parent class
MyClass.Conf(a='Bloom').instantiate()

MyClass(Bloom, 3)

In [8]:
# normal constructor 
my_class = MyClass(a='Bloom', b=999)

# the my_class instance has a .conf attribute
# this is different from .Conf because
# 1) it is an instance variable and
# 2) it is an instantiated dataclass object
# it is the Conf instance that builds this instance of MyClass
my_class.conf

Builds_MyClass(_target_='hydra_zen.funcs.zen_processing', _zen_target='__main__.MyClass', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), _recursive_=False, a='Bloom', b=999, name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_'])

In [9]:
# starting with config
MyClass.Conf(a='Lily').instantiate().conf.instantiate().conf.instantiate()

MyClass(Lily, 3)

In [10]:
# starting with regular constructor
MyClass(a='Lily').conf.instantiate().conf.instantiate().conf.instantiate()

MyClass(Lily, 3)

In [11]:
# editing a config
my_class_conf = MyClass.Conf(a='Lily',b=47)
my_class_conf.a = 'Bloom'
my_class_conf.instantiate()

MyClass(Bloom, 47)

In [12]:
# you can call .conf as well and provide new values for a subset of inputs
my_class = MyClass(a='Lily', b=117)
my_class_bloom = my_class.conf(a="Bloom")
my_class_bloom.instantiate()

MyClass(Bloom, 117)

## wrap an existing class

In [13]:
from sklearn.linear_model import LogisticRegression
WrappedLogisticRegression = add_conf(LogisticRegression)

In [14]:
# conf attr has fully populated fields
WrappedLogisticRegression.Conf().show()

_zen_target: sklearn.linear_model._logistic.LogisticRegression
penalty: l2
dual: false
tol: 0.0001
C: 1.0
fit_intercept: true
intercept_scaling: 1
solver: lbfgs
max_iter: 100
multi_class: auto
verbose: 0
warm_start: false



In [15]:
# can be used to instantiate
WrappedLogisticRegression.Conf(penalty='l1', C=1.0).instantiate()

LogisticRegression(penalty='l1')

In [16]:
# another example
from torch import nn
LazyLinear = add_conf(nn.LazyLinear)
LazyLinear.Conf(out_features=2).instantiate()

/home/ubuntu/anaconda3/envs/orange/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


LazyLinear(in_features=0, out_features=2, bias=True)

In [17]:
# can overwrite imports, this might be sus but could be useful for backwards compatibility
LogisticRegression = add_conf(LogisticRegression)
LogisticRegression.Conf()

Builds_LogisticRegression(_target_='hydra_zen.funcs.zen_processing', _zen_target='sklearn.linear_model._logistic.LogisticRegression', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None, name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_'])

In [18]:
# idempotent
LogisticRegression = add_conf(add_conf(add_conf(add_conf(add_conf(LogisticRegression)))))

## working with functions

In [19]:
# works the same as above - gives function a .conf attribute
# does not work with positional arguments - you need some kind of default value, even if it is None or MISSING
# by default this returns a partial
@add_func_conf
def add_numbers(a:int=MISSING, b:int=MISSING):
    return a + b

In [20]:
add_numbers.Conf(a=1, b=2).instantiate()()

3

In [21]:
# add_func_conf lets you change settings being used in hydra_zen.builds under the hood
# here we make it so that the function is not wrapped in a partial
@add_func_conf(zen_partial=False)
def make_dataframe():
    return pd.DataFrame({'a': 1}, index=[0])

In [22]:
# now instantiation just immediately returns the dataframe
make_dataframe.Conf().instantiate()

,a
0,1


## creating and storing named configurations

In [23]:
@add_conf
class TopLevelClass:
    def __init__(
        self,
        a:int = 0,
        lower_level_class: Any = MISSING,
        **kwargs,
        ):
        self.a = a
        self.lower_level_class = lower_level_class
        
    def __repr__(self):
        return f'TopLevelClass({self.a}, {self.lower_level_class})'
        
@add_conf        
class LowerLevelClass:
    def __init__(
        self,
        b: str = 'hello',
        **kwargs,
    ):
        self.b = b

    def __repr__(self):
        return f'LowerLevelClass({self.b})'

In [24]:
# decorated classes will accept name_, group_, and defaults as arguments
lower_level_class = LowerLevelClass(
    name_='lower_level_class_1',
    )
lower_level_class

LowerLevelClass(hello)

In [25]:
# constructing a decorated class with a name_ argument will automatically store it in ConfigStore
cs.list('')

['_dummy_empty_config_.yaml', 'hydra', 'lower_level_class_1.yaml']

In [26]:
# constructing the .conf dataclass with a name_ argument will also automatically store it in ConfigStore
LowerLevelClass.Conf(
    name_='lower_level_class_2',
    b=99,
    )

Builds_LowerLevelClass(_target_='hydra_zen.funcs.zen_processing', _zen_target='__main__.LowerLevelClass', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), b=99, name_='lower_level_class_2', group_=None, package_='_group_', provider_=None, defaults=['_self_'])

In [27]:
cs.list('')

['_dummy_empty_config_.yaml',
 'hydra',
 'lower_level_class_1.yaml',
 'lower_level_class_2.yaml']

In [28]:
# config group names can be set in the same manner
LowerLevelClass(
    name_='lower_level_class_3',
    group_='lower_level_class',
    )

LowerLevelClass.Conf(
    name_='lower_level_class_4',
    group_='lower_level_class',
    b=99,
    )

Builds_LowerLevelClass(_target_='hydra_zen.funcs.zen_processing', _zen_target='__main__.LowerLevelClass', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), b=99, name_='lower_level_class_4', group_='lower_level_class', package_='_group_', provider_=None, defaults=['_self_'])

In [29]:
LowerLevelClass(
    name_='lower_level_class_3',
    group_='lower_level_class',
    ).conf

Builds_LowerLevelClass(_target_='hydra_zen.funcs.zen_processing', _zen_target='__main__.LowerLevelClass', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), b='hello', name_='lower_level_class_3', group_='lower_level_class', package_='_group_', provider_=None, defaults=['_self_'])

In [30]:
# as can the defaults list
top_level_class_config = TopLevelClass.Conf(
    name_='top_level_class', 
    defaults=['_self_', {'lower_level_class': 'lower_level_class_3'}]
    )

In [31]:
out = launch(top_level_class_config, launch_and_instantiate)
out.return_value

/home/ubuntu/anaconda3/envs/orange/lib/python3.9/site-packages/hydra_zen/_launch.py:235: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(
/home/ubuntu/anaconda3/envs/orange/lib/python3.9/site-packages/hydra/core/default_element.py:124: UserWarning: In 'lower_level_class/lower_level_class_3': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/home/ubuntu/anaconda3/envs/orange/lib/python3.9/site-packages/hydra_zen/_launch.py:265: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  job = run_job(


TopLevelClass(0, LowerLevelClass(hello))

In [32]:
out = launch(
    top_level_class_config, 
    launch_and_instantiate, 
    overrides=['lower_level_class=lower_level_class_4'])
out.return_value

/home/ubuntu/anaconda3/envs/orange/lib/python3.9/site-packages/hydra_zen/_launch.py:235: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(
/home/ubuntu/anaconda3/envs/orange/lib/python3.9/site-packages/hydra/core/default_element.py:124: UserWarning: In 'lower_level_class/lower_level_class_4': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(


TopLevelClass(0, LowerLevelClass(99))

### let's look at more complex configurations

In [33]:
@add_conf
class TopLevelClass:
    def __init__(
        self,
        a:int = 0,
        mid_level_class: Any = MISSING,
        **kwargs,
    ):
        self.a = a
        self.mid_level_class = mid_level_class
        
    def __repr__(self):
        return f'TopLevelClass({self.a}, {self.mid_level_class})'

@add_conf(group_='mlc_group')   
class MidLevelClass:
    def __init__(
        self,
        b:int = 0,
        lower_level_class: Any = MISSING
    ):
        self.b = b
        self.lower_level_class = lower_level_class

    def __repr__(self):
        return f'MidLevelClass({self.b}, {self.lower_level_class})'
        
@add_conf(group_='llc_group')        
class LowerLevelClass:
    def __init__(
        self,
        c: str = 'hello',
        **kwargs,
    ):
        self.c = c

    def __repr__(self):
        return f'LowerLevelClass({self.c})'

In [34]:
llc_1 = LowerLevelClass.Conf(
    c = 'type 1',
    name_='llc_1'
)

llc_2 = LowerLevelClass.Conf(
    c = 'type 2',
    name_='llc_2'
)

llc_3 = LowerLevelClass.Conf(
    c = 'type 3',
    name_='llc_3'
)

llc_4 = LowerLevelClass.Conf(
    c = 'type 4',
    name_='llc_4'
)

llc_1

Builds_LowerLevelClass(_target_='hydra_zen.funcs.zen_processing', _zen_target='__main__.LowerLevelClass', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), c='type 1', name_='llc_1', group_='llc_group', package_='_group_', provider_=None, defaults=['_self_'])

In [45]:
mlc_1 = MidLevelClass.Conf(
    b = 1,
    lower_level_class=MISSING,
    name_='mlc_1',
    defaults=[
            {'/llc_group@lower_level_class': 'llc_3'},
        ]
)

mlc_2 = MidLevelClass.Conf(
    b = 2,
    lower_level_class=MISSING,
    name_='mlc_2'
)

mlc_3 = MidLevelClass.Conf(
    b = 3,
    lower_level_class=MISSING,
    name_='mlc_3'
)

mlc_4 = MidLevelClass.Conf(
    b = 4,
    lower_level_class=llc_4,
    name_='mlc_4'
)

mlc_1.show()

_zen_target: __main__.MidLevelClass
b: 1
lower_level_class:
  _zen_target: __main__.LowerLevelClass
  c: type 3
  name_: llc_3
  group_: llc_group
name_: mlc_1
group_: mlc_group
defaults:
- /llc_group@lower_level_class: llc_3



In [46]:
tlc_1 = TopLevelClass.Conf(
    a = 123,
    mid_level_class=MISSING,
    name_='tlc_1',
    defaults=[
            {
                'mlc_group@mid_level_class': 'mlc_1',
            },
        ]
)

tlc_1.show()

_zen_target: __main__.TopLevelClass
a: 123
mid_level_class: ???
name_: tlc_1
defaults:
- mlc_group@mid_level_class: mlc_1



In [47]:
out = launch(
    tlc_1, 
    launch_and_instantiate
    )
out.return_value

TopLevelClass(123, MidLevelClass(1, LowerLevelClass(type 3)))

In [48]:
out = launch(
    tlc_1, 
    launch_and_instantiate, 
    overrides=['llc_group@mid_level_class.lower_level_class=llc_4']
    )
out.return_value

TopLevelClass(123, MidLevelClass(1, LowerLevelClass(type 3)))

In [39]:
out = launch(
    tlc_1, 
    launch_and_instantiate, 
    overrides=['mid_level_class.lower_level_class.c="type X"']
    )
out.return_value

TopLevelClass(123, MidLevelClass(1, LowerLevelClass(type X)))

In [40]:
out = launch(
    tlc_1, 
    launch_and_instantiate, 
    overrides=['llc_group@lower_level_class=llc_2']
    )
out.return_value

ConfigCompositionException: Could not override 'llc_group@lower_level_class'.
Did you mean to override llc_group@mid_level_class.lower_level_class?
To append to your default list use +llc_group@lower_level_class=llc_2

### using context manager

In [36]:
with ConfMode():
    LowerLevelClass(
        name_='lower_level_class_5',
        group_='lower_level_class',
        b=117,
    )
    top_level_class_config_2 = TopLevelClass(
        a=-9,
        name_='top_level_class_2', 
        defaults=[
            '_self_', 
            {'lower_level_class': 'lower_level_class_5'}]
    )

In [38]:
out = launch(
    top_level_class_config_2, 
    launch_and_instantiate, 
    overrides=['lower_level_class=lower_level_class_4']
    )
out.return_value

TopLevelClass(-9, LowerLevelClass(99))

In [40]:
out = launch(
    top_level_class_config_2, 
    launch_and_instantiate, 
    overrides=['lower_level_class.b=123']
    )
out.return_value

/home/ubuntu/anaconda3/envs/orange/lib/python3.9/site-packages/hydra/core/default_element.py:122: UserWarning: In 'lower_level_class/lower_level_class_5': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(


TopLevelClass(-9, LowerLevelClass(123))

# on something more concrete

In [9]:
from thundercloud.torch.models.components.backbone import LazyBlock, StackedLazyBlocks
from thundercloud.torch.models.model import GenericModel
from typing import Optional, List, Dict, Union
from torch import nn
from functools import partial
from hydra_zen import instantiate
from collections import OrderedDict
from pprint import pprint

In [10]:
# setup conf for torch primitives
Linear = add_conf(nn.Linear)
LazyLinear = add_conf(nn.LazyLinear)
ReLU = add_conf(nn.ReLU)
Dropout = add_conf(nn.Dropout)
BatchNorm = add_conf(nn.BatchNorm1d)
Identity = add_conf(nn.Identity)
Module = add_conf(nn.Module)

In [89]:
# build sequential wrapper
import functools

    
# @functools.wraps(nn.Sequential, updated=())
@add_func_conf(hydra_recursive=False, zen_partial=False)
def make_sequential(
    components:Union[List, Dict]=None,
    **kwargs,
):
    if isinstance(components, dict):
        components = list(components.values())
    return nn.Sequential(*[instantiate(c) for c in components])

In [90]:
make_sequential.conf

types.Builds_make_sequential

In [91]:
# configure
with ConfMode():
    sequential = make_sequential(
        components=[
            Linear(in_features=1, out_features=2), 
            Linear(in_features=2, out_features=1)
            ]
        )

In [92]:
sequential

Builds_make_sequential(_target_='hydra_zen.funcs.zen_processing', _zen_target='__main__.make_sequential', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), _recursive_=False, components=[Builds_Linear(_target_='hydra_zen.funcs.zen_processing', _zen_target='torch.nn.modules.linear.Linear', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), in_features=1, out_features=2, bias=True, device=None, dtype=None, name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_']), Builds_Linear(_target_='hydra_zen.funcs.zen_processing', _zen_target='torch.nn.modules.linear.Linear', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), in_features=2, out_features=1, bias=True, device=None, dtype=None, name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_'])], name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_'])

In [93]:
instantiate(sequential.components[0])

Linear(in_features=1, out_features=2, bias=True)

In [94]:
sequential.instantiate()

Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=1, bias=True)
)

In [95]:
# do overrides
launch(sequential, launch_and_instantiate, overrides=['components.0.in_features=3']).return_value

Sequential(
  (0): Linear(in_features=3, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=1, bias=True)
)

In [15]:
# class based sequential that is recursive
from thundercloud.torch.models.components.backbone import Sequential

with ConfMode():
    my_sequence = Sequential(
        components=[
            Linear(in_features=1, out_features=2), 
            Linear(in_features=2, out_features=1)
        ]
    )

In [17]:
my_sequence

Builds_Sequential(_target_='hydra_zen.funcs.zen_processing', _zen_target='thundercloud.torch.models.components.backbone.Sequential', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), components=[Builds_Linear(_target_='hydra_zen.funcs.zen_processing', _zen_target='torch.nn.modules.linear.Linear', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), in_features=1, out_features=2, bias=True, device=None, dtype=None, name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_']), Builds_Linear(_target_='hydra_zen.funcs.zen_processing', _zen_target='torch.nn.modules.linear.Linear', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), in_features=2, out_features=1, bias=True, device=None, dtype=None, name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_'])], name_=None, group_=None, package_='_group_', provider_=None, defaults=['_self_'])

In [19]:
my_sequence = my_sequence.instantiate()
my_sequence

Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=1, bias=True)
)

## rebuild lazy block

In [178]:
# rebuild lazy block

with ConfMode():

    dropout = Dropout(p=0., name_='dropout')
    
    linear = LazyLinear(
        out_features=1, 
        group_='linear', 
        name_='LazyLinear',
        )
    
    @add_func_conf(hydra_recursive=False, zen_partial=False)
    def make_lazy_block(
            out_features:int = 1,
            input_dropout: Optional[Any] = dropout,
            linear=linear,
            activation: Optional[Any] = MISSING,
            output_dropout: Optional[Any] = dropout,
            batch_norm: Optional[Any] = MISSING,
            **kwargs,
        ):
            net = nn.Sequential()
            if input_dropout is not None:
                net.append(instantiate(input_dropout))
            net.append(instantiate(linear, out_features=out_features))
            if activation is not None:
                net.append(instantiate(activation))
            if output_dropout is not None:
                net.append(instantiate(output_dropout))
            if batch_norm is not None:
                net.append(instantiate(batch_norm, num_features=out_features))
            return net
    

    
    # create a bunch of config options
    
    batch_norm = BatchNorm(
        num_features=1, 
        group_='batch_norm', 
        name_='BatchNorm',
        )
    
    # activations 
    identity = Identity(group_='activation', name_='Identity')
    relu = ReLU(group_='activation', name_='ReLU')

    # set defaults list
    zen_lazy_block = make_lazy_block(
        defaults=[
            '_self_',
            'linear/LazyLinear',
            {'activation': 'Identity'},
            'batch_norm/BatchNorm',
        ]
        )

In [179]:
zen_lazy_block

Builds_make_lazy_block(_target_='hydra_zen.funcs.zen_processing', _zen_target='__main__.make_lazy_block', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), _recursive_=False, out_features=1, input_dropout=Builds_Dropout(_target_='hydra_zen.funcs.zen_processing', _zen_target='torch.nn.modules.dropout.Dropout', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), p=0.0, inplace=False, name_='dropout', group_=None, package_='_group_', provider_=None, defaults=['_self_']), linear=Builds_LazyLinear(_target_='hydra_zen.funcs.zen_processing', _zen_target='torch.nn.modules.linear.LazyLinear', _zen_exclude=('name_', 'group_', 'package_', 'provider_', 'defaults'), out_features=1, bias=True, device=None, dtype=None, name_='LazyLinear', group_='linear', package_='_group_', provider_=None, defaults=['_self_']), activation='???', output_dropout=Builds_Dropout(_target_='hydra_zen.funcs.zen_processing', _zen_target='torch.nn.modules.dropout.Dropout', _zen_exclude

In [180]:
launch(
    zen_lazy_block, 
    launch_and_instantiate, 
    overrides=[
        'out_features=8', 
        'activation=ReLU', 
        'input_dropout.p=0.2'
        ],
    ).return_value

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): LazyLinear(in_features=0, out_features=8, bias=True)
  (2): ReLU()
  (3): Dropout(p=0.0, inplace=False)
  (4): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [2]:
# we've also implemented a class based LazyBlock
# we can use it in combo with our Sequential wrapper to demonstrate using a base config that you can alter

from thundercloud.torch.models.components.backbone import LazyBlock,LazyLinear,Dropout,ReLU,BatchNorm, Sequential

with ConfMode():

    # this is a partial conf, missing out_features
    lazy_block_base = LazyBlock(
        activation = ReLU(),
        output_dropout = Dropout(p=0., name_='dropout'),
        batch_norm = BatchNorm()
    )

    # modify the base conf for each element in the list
    my_sequential = Sequential(
        components = [
            lazy_block_base(out_features = 256,input_dropout = Dropout(p=0.2, name_='dropout')),
            lazy_block_base(out_features = 128),
            lazy_block_base(out_features = 64)
        ]
    )

In [4]:
my_sequential.show()

_zen_target: thundercloud.torch.models.components.backbone.Sequential
components:
- _zen_target: thundercloud.torch.models.components.backbone.LazyBlock
  out_features: 256
  input_dropout:
    _zen_target: torch.nn.modules.dropout.Dropout
    p: 0.2
    inplace: false
    name_: dropout
  activation:
    _zen_target: torch.nn.modules.activation.ReLU
    inplace: false
  output_dropout:
    _zen_target: torch.nn.modules.dropout.Dropout
    p: 0.0
    inplace: false
    name_: dropout
  batch_norm:
    _zen_target: torch.nn.modules.batchnorm.BatchNorm1d
    num_features: 1
    eps: 1.0e-05
    momentum: 0.1
    affine: true
    track_running_stats: true
    device: null
    dtype: null
- _zen_target: thundercloud.torch.models.components.backbone.LazyBlock
  out_features: 128
  input_dropout: null
  activation:
    _zen_target: torch.nn.modules.activation.ReLU
    inplace: false
  output_dropout:
    _zen_target: torch.nn.modules.dropout.Dropout
    p: 0.0
    inplace: false
    name_: d

In [5]:
my_sequential = my_sequential.instantiate()
my_sequential

/home/ubuntu/anaconda3/envs/orange/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Sequential(
  (0): LazyBlock(
    (0): Dropout(p=0.2, inplace=False)
    (1): LazyLinear(in_features=0, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): LazyBlock(
    (0): LazyLinear(in_features=0, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.0, inplace=False)
    (3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): LazyBlock(
    (0): LazyLinear(in_features=0, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.0, inplace=False)
    (3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [182]:
# back to working with the function versions

with ConfMode():
    relu_zen_block = make_lazy_block(
        activation=relu,
        batch_norm=batch_norm,
        )
    sequential = make_sequential(
        components=[
            relu_zen_block,
            relu_zen_block,
            relu_zen_block,
            ]
        )

In [184]:
instantiated_sequential = launch(
    sequential, 
    launch_and_instantiate, 
    # overrides=['components.0.out_features=6',],
    ).return_value

In [185]:
instantiated_sequential

Sequential(
  (0): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): LazyLinear(in_features=0, out_features=1, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
    (4): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): LazyLinear(in_features=0, out_features=1, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
    (4): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): LazyLinear(in_features=0, out_features=1, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.0, inplace=False)
    (4): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [ ]:
class TimesX:
    def __init__(self,x=1.0):
        self.x = x
    
    def times(self,y=1.0):
        return self.x*y

class Times2X(TimesX):
    def __init__(self,x=1.0):
        super().__init__(2*x)
